## Класические модели с TfIdf

In [3]:
import os
import pandas as pd
import numpy as np

In [4]:
paths = {
    'MODELS':'models',
    'LR':os.path.join('models','LogisticRegression'),
    'FASTTEXT':os.path.join('models','fasttext'),
    'DATA':'data',
    'TRAIN':os.path.join('data','train'),
    'TEST':os.path.join('data','test')
}

### Загрузка предобработанных данных

In [36]:
df_gensim = pd.read_csv(os.path.join(paths['DATA'],'gensim.csv'), index_col=0).dropna()

In [37]:
df_cleaner = pd.read_csv(os.path.join(paths['DATA'],'cleaner.csv'), index_col=0).dropna()

In [38]:
df_gensim.head(5)

,text,topic
0,как стало известно агентству ассошиэйтед пресс...,Россия
1,зале игровых автоматов третьем ярусе подземног...,Россия
2,япония приняла решение разморозить кредиты рос...,Россия
3,британцы отмечают сегодня скорбную дату вторую...,Мир
4,понедельник директор департамента внешних связ...,Россия


In [39]:
df_cleaner.head(5)

,text,topic
0,как стало известно агентству ассошиэйтед пресс...,Россия
1,зале игровых автоматов третьем ярусе подземног...,Россия
2,япония приняла решение разморозить кредиты рос...,Россия
3,британцы отмечают сегодня скорбную дату вторую...,Мир
4,понедельник директор департамента внешних связ...,Россия


### Формирование тестовой и обучающей выборки с учетом несбалансированности классов

In [40]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [41]:
X_train_gensim, X_test_gensim, y_train_gensim, y_test_gensim = train_test_split(df_gensim.text, 
                                                                                df_gensim.topic,  
                                                                                test_size=0.2, 
                                                                                stratify=df_gensim.topic,
                                                                                random_state=11)

In [42]:
X_train_clean, X_test_clean, y_train_clean, y_test_clean = train_test_split(df_cleaner.text,
                                                                            df_cleaner.topic,
                                                                            test_size=0.2, 
                                                                            stratify=df_cleaner.topic, 
                                                                            random_state=11)

#### TfIdf преобразование для данных преобразованных gensim

In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.linalg import svd

In [47]:
%%time
vectorizer_gensim = TfidfVectorizer()
X_train_gensim_tfidf = vectorizer_gensim.fit_transform(X_train_gensim.values)
X_test_gensim_tfidf = vectorizer_gensim.transform(X_test_gensim)

CPU times: total: 1min 51s
Wall time: 1min 51s


Понижение признакового пространства:
1. с помощью матричного разложения
2. с помощью регуляризатора Лассо 

In [48]:
X_train_gensim_tfidf

<589199x922140 sparse matrix of type '<class 'numpy.float64'>'
	with 75113056 stored elements in Compressed Sparse Row format>

In [51]:
from sklearn.decomposition import TruncatedSVD

In [80]:
%%time
modelPCA = TruncatedSVD(n_components=16)
X_train = modelPCA.fit_transform(X=X_train_gensim_tfidf)
X_test = modelPCA.transform(X=X_test_gensim_tfidf)
y_train = y_train_gensim
y_test = y_test_gensim

CPU times: total: 1min 24s
Wall time: 1min 10s


In [58]:
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

In [72]:
%%time
modelLR = LogisticRegression(random_state=11, n_jobs=16, solver='saga')
modelLR.fit(X_train, y_train)
predictLR = modelLR.predict(X_test)
print(classification_report(y_test,predictLR))

                   precision    recall  f1-score   support

           Бизнес       0.56      0.13      0.21      1480
      Бывший_СССР       0.79      0.70      0.74     10680
              Дом       0.84      0.72      0.77      4347
         Из_жизни       0.55      0.46      0.50      5521
   Интернет_и_СМИ       0.70      0.63      0.66      8933
         Культура       0.81      0.84      0.82     10759
              Мир       0.75      0.82      0.78     27324
  Наука_и_техника       0.79      0.81      0.80     10626
      Путешествия       0.69      0.33      0.45      1281
           Россия       0.73      0.81      0.77     32088
Силовые_структуры       0.58      0.32      0.41      3919
            Спорт       0.96      0.95      0.96     12883
         Ценности       0.88      0.72      0.79      1553
        Экономика       0.78      0.82      0.80     15906

         accuracy                           0.77    147300
        macro avg       0.74      0.65      0.68    14

In [85]:
%%time
modelLasso = LogisticRegression(penalty='l1',random_state=11, n_jobs=16, solver='saga')
modelLasso.fit(X_train_gensim_tfidf, y_train)
predictLR = modelLasso.predict(X_test_gensim_tfidf)
print(classification_report(y_test,predictLR))

C:\Users\Alexa\Documents\ML\Work_with_text\predict news\venv-text\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


                   precision    recall  f1-score   support

           Бизнес       0.70      0.47      0.56      1480
      Бывший_СССР       0.86      0.86      0.86     10680
              Дом       0.87      0.84      0.85      4347
         Из_жизни       0.71      0.63      0.66      5521
   Интернет_и_СМИ       0.79      0.75      0.77      8933
         Культура       0.88      0.91      0.89     10759
              Мир       0.82      0.86      0.84     27324
  Наука_и_техника       0.86      0.87      0.87     10626
      Путешествия       0.84      0.69      0.76      1281
           Россия       0.82      0.85      0.84     32088
Силовые_структуры       0.76      0.60      0.67      3919
            Спорт       0.97      0.97      0.97     12883
         Ценности       0.92      0.84      0.88      1553
        Экономика       0.86      0.87      0.87     15906

         accuracy                           0.84    147300
        macro avg       0.83      0.79      0.81    14

In [89]:
coef = modelLasso.coef_


In [90]:
import pickle
    

In [91]:
with open('coef.bin','wb') as file:
    pickle.dump(coef,file)